# Introduction to GPU Computing

In today's computational landscape, GPUs (Graphics Processing Units) have evolved beyond their original purpose of rendering graphics. They have become essential tools for high-performance computing (HPC), capable of performing a vast number of calculations simultaneously. This ability to handle massive parallelism makes GPUs particularly useful for scientific simulations, machine learning, and other computationally intensive tasks.

In this section, we'll explore the basics of GPU computing, including the architecture of GPUs and the programming models that allow us to harness their power. Specifically, we'll focus on two main approaches: CUDA and OpenACC.

## What is GPU Computing?

GPU computing involves using a GPU to perform computations that would otherwise be handled by a CPU. Unlike CPUs, which have a few cores optimized for sequential processing, GPUs have thousands of smaller, more efficient cores designed for parallel tasks. This makes GPUs ideal for workloads that can be broken down into many smaller, independent tasks.

## Why Use GPUs in HPC?

- **Massive Parallelism**: GPUs can execute thousands of threads simultaneously, making them well-suited for parallel algorithms.
- **High Throughput**: With their large number of cores, GPUs can process a significant amount of data in parallel, leading to faster execution times for many applications.
- **Energy Efficiency**: GPUs can often perform more computations per watt compared to CPUs, making them a more energy-efficient choice for large-scale computations.

In the following sections, we will dive into the basics of CUDA and OpenACC, two popular programming models for GPU computing.


## CUDA Basics

CUDA (Compute Unified Device Architecture) is a parallel computing platform and application programming interface (API) model created by NVIDIA. It allows developers to use NVIDIA GPUs for general-purpose processing (an approach known as GPGPU, General-Purpose computing on Graphics Processing Units).

### Key Concepts in CUDA

- **Thread**: The smallest unit of processing. Each thread runs a copy of the kernel (a function that runs on the GPU).
- **Block**: A group of threads that can cooperate amongst themselves via shared memory. Blocks are organized into a grid.
- **Grid**: A collection of blocks. The grid represents the entire execution space for a given kernel.

Let's start by writing a simple CUDA program to perform matrix multiplication, a common operation in many scientific and engineering applications.


## CUDA Basics

CUDA (Compute Unified Device Architecture) is a parallel computing platform and application programming interface (API) model created by NVIDIA. It allows developers to use NVIDIA GPUs for general-purpose processing (an approach known as GPGPU, General-Purpose computing on Graphics Processing Units).

### Key Concepts in CUDA

- **Thread**: The smallest unit of processing. Each thread runs a copy of the kernel (a function that runs on the GPU).
- **Block**: A group of threads that can cooperate amongst themselves via shared memory. Blocks are organized into a grid.
- **Grid**: A collection of blocks. The grid represents the entire execution space for a given kernel.

Let's start by writing a simple CUDA program to perform matrix multiplication, a common operation in many scientific and engineering applications.


In [ ]:
# Check the GPU available in Google Colab
!nvidia-smi


Sun Sep  1 16:54:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Write the CUDA C code for matrix multiplication to a file
cuda_code = """
#include <stdio.h>
#include <cuda_runtime.h>

#define N 512  // Define the size of the matrix

__global__ void matrixMul(float *A, float *B, float *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0;

    if (row < n && col < n) {
        for (int i = 0; i < n; ++i) {
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);

    // Allocate memory on the host
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize matrices A and B with more variation
    for (int i = 0; i < N * N; ++i) {
        int row = i / N;
        int col = i % N;
        h_A[i] = (row + 1) + (col % 5);  // Create variation across rows and columns
        h_B[i] = (col + 1) * (row % 3 + 1);  // Create variation across rows and columns
    }

    // Allocate memory on the device
    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, size);
    cudaMalloc((void **)&d_B, size);
    cudaMalloc((void **)&d_C, size);

    // Copy matrices from host to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Define the block size and grid size
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Launch the matrix multiplication kernel
    matrixMul<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copy the result matrix back to the host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Print the result matrix
    printf("Result matrix C (only showing a part of it):\\n");
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            printf("%f ", h_C[i * N + j]);
        }
        printf("\\n");
    }

    // Free memory on device and host
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}
"""

with open('matrix_mul.cu', 'w') as f:
    f.write(cuda_code)


## Understanding the CUDA Matrix Multiplication Code

This CUDA program performs matrix multiplication, a fundamental operation in many scientific and engineering applications, by leveraging the parallel processing power of GPUs.

- **Matrix Size Definition**: `#define N 512` sets the matrix dimensions to 512x512, which can be adjusted as needed.

- **CUDA Kernel Function (`matrixMul`)**: The kernel function performs the actual multiplication. Each thread computes a single element of the result matrix `C` by iterating over a row of matrix `A` and a column of matrix `B`. The `blockIdx` and `threadIdx` are used to determine the position of the element each thread is responsible for.

- **Memory Management**:
  - **Host Memory**: Memory for matrices `A`, `B`, and `C` is allocated on the host (CPU) using `malloc`.
  - **Device Memory**: Corresponding memory on the GPU is allocated using `cudaMalloc`.
  - **Data Transfer**: The matrices `A` and `B` are copied from the host to the GPU before computation, and the result matrix `C` is copied back from the GPU to the host after the kernel execution.

- **Grid and Block Configuration**: The grid and block dimensions are set using `dim3`, with a 16x16 block size to distribute the workload among the GPU threads efficiently.

- **Kernel Launch**: The `matrixMul` kernel is launched with the specified grid and block dimensions to perform the matrix multiplication on the GPU.

- **Result Output**: After computation, a portion of the result matrix `C` is printed to verify the output.

- **Memory Cleanup**: Finally, both host and device memory are freed to prevent memory leaks.

This example highlights the essential steps in a CUDA program, including memory management, kernel configuration, and parallel computation, providing a foundation for more complex GPU-accelerated applications.


## Running the CUDA Program

Next, we'll compile and run the CUDA program to see how GPU-based matrix multiplication works. The result matrix should reflect the combined computation of the two input matrices.

We'll use `nvcc`, the CUDA compiler, to compile the code and then execute the compiled binary. If everything is set up correctly, you should see the output of the matrix multiplication displayed below.


In [ ]:
# Compile the CUDA program
!nvcc -o matrix_mul matrix_mul.cu



In [ ]:
# Run the CUDA program
!./matrix_mul



Result matrix C (only showing a part of it):
3065.000000 6130.000000 9195.000000 12260.000000 15325.000000 18390.000000 21455.000000 24520.000000 27585.000000 30650.000000 
4088.000000 8176.000000 12264.000000 16352.000000 20440.000000 24528.000000 28616.000000 32704.000000 36792.000000 40880.000000 
5111.000000 10222.000000 15333.000000 20444.000000 25555.000000 30666.000000 35777.000000 40888.000000 45999.000000 51110.000000 
6134.000000 12268.000000 18402.000000 24536.000000 30670.000000 36804.000000 42938.000000 49072.000000 55206.000000 61340.000000 
7157.000000 14314.000000 21471.000000 28628.000000 35785.000000 42942.000000 50099.000000 57256.000000 64413.000000 71570.000000 
8180.000000 16360.000000 24540.000000 32720.000000 40900.000000 49080.000000 57260.000000 65440.000000 73620.000000 81800.000000 
9203.000000 18406.000000 27609.000000 36812.000000 46015.000000 55218.000000 64421.000000 73624.000000 82827.000000 92030.000000 
10226.000000 20452.000000 30678.000000 40904.000

In [ ]:
# Write the CUDA C code for dense and sparse matrix multiplication to a file
cuda_code = """
#include <stdio.h>
#include <cuda_runtime.h>

#define N 512  // Define the size of the matrix

// Kernel for dense matrix multiplication
__global__ void denseMatrixMul(float *A, float *B, float *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0;

    if (row < n && col < n) {
        for (int i = 0; i < n; ++i) {
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

// Kernel for sparse matrix multiplication (simplified for demo purposes)
__global__ void sparseMatrixMul(int *rowPtr, int *colInd, float *values, float *B, float *C, int n, int nnz) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0;

    if (row < n && col < n) {
        for (int idx = rowPtr[row]; idx < rowPtr[row + 1]; ++idx) {
            int j = colInd[idx];
            sum += values[idx] * B[j * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);

    // Allocate memory on the host for dense matrices
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C_dense = (float *)malloc(size);
    float *h_C_sparse = (float *)malloc(size);


    // Initialize dense matrices A and B with varying values
    for (int i = 0; i < N * N; ++i) {
        h_A[i] = ((i % N) + 1) * ((i / N) % 3 + 1);  // Example: values are products of row and column indices with some variation
        h_B[i] = ((i / N) + 1) * 0.5f * ((i % N) % 4 + 1);  // Example: values depend on both row and column indices with multiplication factors
    }


    // Allocate memory on the device for dense matrices
    float *d_A, *d_B, *d_C_dense;
    cudaMalloc((void **)&d_A, size);
    cudaMalloc((void **)&d_B, size);
    cudaMalloc((void **)&d_C_dense, size);

    // Copy matrices from host to device for dense multiplication
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Define the block size and grid size for dense multiplication
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Launch the dense matrix multiplication kernel
    denseMatrixMul<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C_dense, N);

    // Copy the result matrix back to the host for dense multiplication
    cudaMemcpy(h_C_dense, d_C_dense, size, cudaMemcpyDeviceToHost);

    // Print results for dense matrix multiplication
    printf("Result matrix C (Dense - showing a part of it):\\n");
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            printf("%f ", h_C_dense[i * N + j]);
        }
        printf("\\n");
    }

    // Example of sparse matrix multiplication
    // Note: Sparse matrix in CSR format is represented by rowPtr, colInd, and values arrays
    int nnz = 3 * N;  // Example: 3 non-zero entries per row
    int *h_rowPtr = (int *)malloc((N + 1) * sizeof(int));
    int *h_colInd = (int *)malloc(nnz * sizeof(int));
    float *h_values = (float *)malloc(nnz * sizeof(float));


    // Initialize sparse matrix A in CSR format
    for (int i = 0; i < N; ++i) {
        h_rowPtr[i] = i * 3;  // 3 non-zeros per row
        for (int j = 0; j < 3; ++j) {
            h_colInd[i * 3 + j] = (i + j) % N;  // Example column indices
            h_values[i * 3 + j] = 1.0f;  // Non-zero values
        }
    }
    h_rowPtr[N] = nnz;  // Last entry of rowPtr

    // Allocate memory on the device for sparse matrix multiplication
    int *d_rowPtr, *d_colInd;
    float *d_values, *d_C_sparse;
    cudaMalloc((void **)&d_rowPtr, (N + 1) * sizeof(int));
    cudaMalloc((void **)&d_colInd, nnz * sizeof(int));
    cudaMalloc((void **)&d_values, nnz * sizeof(float));
    cudaMalloc((void **)&d_C_sparse, size);

    // Copy sparse matrix data from host to device
    cudaMemcpy(d_rowPtr, h_rowPtr, (N + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colInd, h_colInd, nnz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, h_values, nnz * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);  // Reuse B from dense multiplication

    // Launch the sparse matrix multiplication kernel
    sparseMatrixMul<<<blocksPerGrid, threadsPerBlock>>>(d_rowPtr, d_colInd, d_values, d_B, d_C_sparse, N, nnz);

    // Copy the result matrix back to the host for sparse multiplication
    cudaMemcpy(h_C_sparse, d_C_sparse, size, cudaMemcpyDeviceToHost);

    // Print results for sparse matrix multiplication
    printf("\\nResult matrix C (Sparse - showing a part of it):\\n");
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            printf("%f ", h_C_sparse[i * N + j]);
        }
        printf("\\n");
    }

    // Free memory on device and host
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C_dense);
    cudaFree(d_rowPtr);
    cudaFree(d_colInd);
    cudaFree(d_values);
    cudaFree(d_C_sparse);
    free(h_A);
    free(h_B);
    free(h_C_dense);
    free(h_C_sparse);
    free(h_rowPtr);
    free(h_colInd);
    free(h_values);

    return 0;
}
"""

with open('matrix_mul.cu', 'w') as f:
    f.write(cuda_code)


## Understanding the CUDA Code for Dense and Sparse Matrix Multiplication

This CUDA program demonstrates two different approaches to matrix multiplication: dense and sparse. Matrix multiplication is a core operation in many high-performance computing applications, and understanding both dense and sparse implementations is crucial for optimizing performance depending on the data characteristics.

### Dense Matrix Multiplication
- **Kernel Function (`denseMatrixMul`)**: This kernel computes the product of two dense matrices `A` and `B`, storing the result in matrix `C`. Each thread is responsible for calculating a single element in the result matrix `C` by iterating over a row in `A` and a column in `B`.
- **Host and Device Memory Management**:
  - **Host Memory**: Allocated for matrices `A`, `B`, and `C_dense` using `malloc`.
  - **Device Memory**: Corresponding memory is allocated on the GPU using `cudaMalloc`.
  - **Data Transfer**: Matrices `A` and `B` are copied from host to device memory before the kernel is executed, and the resulting matrix `C_dense` is copied back to the host after the kernel execution.
- **Kernel Launch Configuration**: The matrix multiplication kernel is launched with a grid and block configuration determined by the size of the matrix and the number of threads per block, maximizing GPU resource utilization.

### Sparse Matrix Multiplication
- **Sparse Matrix Representation**: Sparse matrices store only non-zero elements to save memory and computation time. Here, the sparse matrix `A` is represented in Compressed Sparse Row (CSR) format using three arrays: `rowPtr`, `colInd`, and `values`.
  - **`rowPtr`**: Indicates the start and end of each row in the `colInd` and `values` arrays.
  - **`colInd`**: Stores the column indices of the non-zero elements.
  - **`values`**: Stores the actual non-zero values of the matrix.
- **Kernel Function (`sparseMatrixMul`)**: This kernel performs matrix multiplication using the sparse matrix `A` and dense matrix `B`, storing the result in matrix `C_sparse`. Each thread computes a single element in `C_sparse` by accessing only the non-zero elements in `A`.
- **Host and Device Memory Management**:
  - **Host Memory**: Additional memory is allocated for the sparse matrix representation (`rowPtr`, `colInd`, and `values`).
  - **Device Memory**: Memory for the sparse matrix components and the result matrix `C_sparse` is allocated on the GPU.
  - **Data Transfer**: Sparse matrix data is copied from host to device before kernel execution, and the resulting matrix `C_sparse` is copied back to the host afterward.

### Result Verification and Output
- After executing both the dense and sparse matrix multiplication kernels, the program prints a portion of the resulting matrices `C_dense` and `C_sparse`. This output allows for verification of the correctness of the matrix multiplication operations.

### Memory Cleanup
- The program ensures proper cleanup by freeing both host and device memory after the computation, preventing memory leaks and ensuring efficient use of resources.

This code provides a practical example of how to implement and compare dense and sparse matrix multiplication on a GPU using CUDA, demonstrating the advantages of using sparse matrices when dealing with data that contains many zeroes.


In [ ]:
# Compile the CUDA program
!nvcc -lcusparse -o matrix_mul matrix_mul.cu


In [ ]:
# Run the CUDA program
!./matrix_mul


Result matrix C (Dense - showing a part of it):
22435164.000000 44870328.000000 67305608.000000 89740656.000000 22435164.000000 44870328.000000 67305608.000000 89740656.000000 22435164.000000 44870328.000000 
44870328.000000 89740656.000000 134611216.000000 179481312.000000 44870328.000000 89740656.000000 134611216.000000 179481312.000000 44870328.000000 89740656.000000 
67305608.000000 134611216.000000 201916704.000000 269222432.000000 67305608.000000 134611216.000000 201916704.000000 269222432.000000 67305608.000000 134611216.000000 
22435164.000000 44870328.000000 67305608.000000 89740656.000000 22435164.000000 44870328.000000 67305608.000000 89740656.000000 22435164.000000 44870328.000000 
44870328.000000 89740656.000000 134611216.000000 179481312.000000 44870328.000000 89740656.000000 134611216.000000 179481312.000000 44870328.000000 89740656.000000 
67305608.000000 134611216.000000 201916704.000000 269222432.000000 67305608.000000 134611216.000000 201916704.000000 269222432.00000

# OpenACC

## Introduction to OpenACC

While CUDA is a powerful and flexible tool for GPU programming, it requires detailed management of the GPU's memory and parallelization. OpenACC is a higher-level programming model designed to simplify the process of parallelizing code. With OpenACC, you can accelerate your existing C, C++, and Fortran applications without requiring in-depth knowledge of GPU architecture.

### Key Concepts in OpenACC

- **Directives**: OpenACC uses compiler directives (pragmas) to specify which parts of the code should run on the GPU. This allows for incremental parallelization of existing code.
- **Parallel Region**: A block of code that is executed by multiple threads in parallel on the GPU.
- **Data Region**: Specifies the movement of data between the CPU and GPU memory.

Let's take a look at a similar matrix multiplication example using OpenACC.


In [ ]:
# Install wget and the necessary dependencies
!apt-get install -y wget build-essential

# Download the NVIDIA HPC SDK
!wget https://developer.download.nvidia.com/hpc-sdk/23.1/nvhpc_2023_231_Linux_x86_64_cuda_12.0.tar.gz -O nvhpc.tar.gz

# Extract the downloaded tarball
!tar -xzf nvhpc.tar.gz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2024-09-01 17:20:34--  https://developer.download.nvidia.com/hpc-sdk/23.1/nvhpc_2023_231_Linux_x86_64_cuda_12.0.tar.gz
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5084055249 (4.7G) [application/x-gzip]
Saving to: ‘nvhpc.tar.gz’

nvhpc.tar.gz        100%[===================>]   4.73G  64.0MB/s    in 76s     

2024-09-01 17:21:50 (64.1 MB/s) - ‘nvhpc.tar.gz’ saved [5084055249/5084055249]



In [ ]:
# Navigate to the extracted folder and run the install script non-interactively
!cd nvhpc_2023_231_Linux_x86_64_cuda_12.0 && yes "" | ./install --installpath /usr/local/nvhpc --accept --silent



Welcome to the NVIDIA HPC SDK Linux installer!

You are installing NVIDIA HPC SDK 2023 version 23.1 for Linux_x86_64.
Please note that all Trademarks and Marks are the properties
of their respective owners.

Press enter to continue...

A network installation will save disk space by having only one copy of the
compilers and most of the libraries for all compilers on the network, and
the main installation needs to be done once for all systems on the network.

1  Single system install
2  Network install

Please choose install option: 

Please specify the directory path under which the software will be installed.
The default directory is /opt/nvidia/hpc_sdk, but you may install anywhere you wish,
assuming you have permission to do so.

Installation directory? [/opt/nvidia/hpc_sdk] 

Note: directory /opt/nvidia/hpc_sdk was created.

Installing NVIDIA HPC SDK version 23.1 into /opt/nvidia/hpc_sdk
Making symbolic links in /opt/nvidia/hpc_sdk/Linux_x86_64/2023

generating environment modules 

In [ ]:
import os
# Set up environment variables for the NVIDIA HPC SDK installed in /opt/nvidia/hpc_sdk
os.environ['PATH'] = '/opt/nvidia/hpc_sdk/Linux_x86_64/23.1/compilers/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/opt/nvidia/hpc_sdk/Linux_x86_64/23.1/compilers/lib:' + os.environ['LD_LIBRARY_PATH']


In [ ]:
# Check if nvc is in the PATH
!which nvc


/opt/nvidia/hpc_sdk/Linux_x86_64/23.1/compilers/bin/nvc


In [ ]:
openacc_code = """
#include <stdio.h>
#include <stdlib.h>

#define N 512  // Define the size of the matrix

int main() {
    int size = N * N * sizeof(float);

    // Allocate memory on the host
    float *A = (float *)malloc(size);
    float *B = (float *)malloc(size);
    float *C = (float *)malloc(size);

    // Initialize matrices A and B with varying values
    for (int i = 0; i < N * N; i++) {
        int row = i / N;
        int col = i % N;
        A[i] = (float)((row + 1) * (col + 2)) * 0.5f;  // Initialize A with values based on row and column
        B[i] = (float)((col + 1) * (row + 2)) * 0.3f;  // Initialize B with values based on row and column
    }

    // Perform matrix multiplication using OpenACC
    #pragma acc data copyin(A[0:N*N], B[0:N*N]), copyout(C[0:N*N])
    {
        #pragma acc parallel loop collapse(2)
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                float sum = 0.0f;
                for (int k = 0; k < N; k++) {
                    sum += A[i * N + k] * B[k * N + j];
                }
                C[i * N + j] = sum;
            }
        }
    }

    // Print a portion of the result matrix
    printf("Result matrix C (only showing a part of it):\\n");
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            printf("%f ", C[i * N + j]);
        }
        printf("\\n");
    }

    // Free memory
    free(A);
    free(B);
    free(C);

    return 0;
}
"""

# Save the OpenACC code to a file
with open('matrix_mul_openacc.c', 'w') as f:
    f.write(openacc_code)


## Understanding the OpenACC Code for Matrix Multiplication

This OpenACC program demonstrates matrix multiplication, a fundamental operation in many scientific computing tasks, using GPU acceleration. OpenACC is a user-friendly directive-based programming model that allows developers to parallelize code easily and target accelerators like GPUs without deep knowledge of GPU architecture.

### Matrix Initialization
- **Matrix Size and Memory Allocation**:
  - The matrix size `N` is defined as 512, resulting in matrices `A`, `B`, and `C` of size 512x512.
  - Memory for these matrices is allocated on the host using `malloc`, with each matrix occupying `N*N*sizeof(float)` bytes.
  
- **Initializing Matrices**:
  - Matrices `A` and `B` are initialized with varying values based on their row and column indices. This variation ensures that each element in the matrices has a unique value, allowing for meaningful computation and results.
  - Matrix `A` is filled with values calculated as `((row + 1) * (col + 2)) * 0.5f`.
  - Matrix `B` is initialized with values calculated as `((col + 1) * (row + 2)) * 0.3f`.

### Parallel Matrix Multiplication with OpenACC
- **OpenACC Directives**:
  - The `#pragma acc data` directive is used to manage data transfer between the host and the device (GPU). It ensures that matrices `A` and `B` are copied to the device before the computation, and matrix `C` is copied back to the host after the computation.
  - The `#pragma acc parallel loop collapse(2)` directive specifies that the following nested loops should be parallelized, allowing multiple threads on the GPU to perform the matrix multiplication concurrently. The `collapse(2)` clause indicates that both loops should be parallelized together, enabling efficient use of the GPU's parallel architecture.

- **Matrix Multiplication**:
  - The nested loops iterate over the rows of `A` and columns of `B` to compute each element of the result matrix `C`. Each element `C[i * N + j]` is calculated as the dot product of the `i-th` row of `A` and the `j-th` column of `B`.
  - The inner loop accumulates the product of corresponding elements from `A` and `B`, storing the sum in `C`.

### Result Display and Memory Cleanup
- **Output**:
  - After computation, the program prints a portion of the result matrix `C` (the top-left 10x10 submatrix). This partial display allows for quick verification of the computation's correctness.
  
- **Memory Management**:
  - The program concludes by freeing the allocated memory for matrices `A`, `B`, and `C` on the host, ensuring efficient use of resources and preventing memory leaks.

This code provides a practical example of how to use OpenACC for parallelizing a common computational task (matrix multiplication) and demonstrates how OpenACC simplifies the process of leveraging GPU acceleration.


## Running the OpenACC Program

Now that we've written our OpenACC code for matrix multiplication, let's compile and run it. The OpenACC program should produce a result matrix similar to the CUDA version but with potentially simpler code.

OpenACC abstracts much of the complexity of GPU programming, making it easier to parallelize existing C or Fortran code. Let's see how it performs.


In [ ]:
# Compile the OpenACC code using nvc (NVIDIA C Compiler) from the HPC SDK
!nvc -acc -o matrix_mul_openacc matrix_mul_openacc.c -Minfo=accel


main:
     25, Generating copyin(A[:262144]) [if not already present]
         Generating copyout(C[:262144]) [if not already present]
         Generating copyin(B[:262144]) [if not already present]
         Generating NVIDIA GPU code
         27, #pragma acc loop gang collapse(2) /* blockIdx.x */
         28,   /* blockIdx.x collapsed */
         30, #pragma acc loop vector(128) /* threadIdx.x */
             Generating implicit reduction(+:sum)
     30, Loop is parallelizable


In [ ]:
# Run the OpenACC program
!./matrix_mul_openacc


Result matrix C (only showing a part of it):
6770035.000000 13540070.000000 20310106.000000 27080140.000000 33850176.000000 40620212.000000 47390248.000000 54160280.000000 60930320.000000 67700352.000000 
13540070.000000 27080140.000000 40620212.000000 54160280.000000 67700352.000000 81240424.000000 94780496.000000 108320560.000000 121860640.000000 135400704.000000 
20310108.000000 40620216.000000 60930320.000000 81240432.000000 101550528.000000 121860640.000000 142170736.000000 162480864.000000 182790944.000000 203101056.000000 
27080140.000000 54160280.000000 81240424.000000 108320560.000000 135400704.000000 162480848.000000 189560992.000000 216641120.000000 243721280.000000 270801408.000000 
33850176.000000 67700352.000000 101550528.000000 135400704.000000 169250880.000000 203101056.000000 236951232.000000 270801408.000000 304651584.000000 338501760.000000 
40620216.000000 81240432.000000 121860640.000000 162480864.000000 203101056.000000 243721280.000000 284341472.000000 324961728.

## Comparing CUDA and OpenACC

Both CUDA and OpenACC are powerful tools for leveraging the computational power of GPUs, but they serve different purposes and target different audiences:

- **CUDA**: Offers more control over the hardware and is ideal for developers who need fine-grained optimization and are familiar with GPU architecture. It requires manual management of memory and parallelization, but this can lead to highly optimized code.

- **OpenACC**: Provides a higher-level, more abstracted approach to GPU programming. It's easier to use for those who want to accelerate their applications without diving deeply into the specifics of GPU hardware. OpenACC is often used to incrementally parallelize existing applications.

### Key Takeaways

- **Flexibility vs. Ease of Use**: CUDA is more flexible but requires more effort, while OpenACC is easier to use but might not offer the same level of optimization.
- **Learning Curve**: CUDA has a steeper learning curve compared to OpenACC.
- **Performance**: Depending on the application and how well the code is optimized, CUDA might offer better performance, but OpenACC can still deliver significant speedups with much less effort.

### Conclusion

In this session, we explored the basics of GPU computing, focusing on two popular approaches: CUDA and OpenACC. We've implemented matrix multiplication in both frameworks and compared their usage. Understanding both CUDA and OpenACC allows you to choose the best tool for your specific needs, whether that’s maximum performance or ease of development.

Continue experimenting with these examples and explore how you can leverage GPU computing for your projects!
